# Unit 2 Exercise - Neural Network Implementation
## Team: Kirk Henrich Gamo & Dallas Aquino

This notebook contains solutions to neural network problems completed by our team:

### **Problem 1: Dense Layer Implementation (50 points)** - *Completed by Kirk and Dallas*
- Develop a Dense_Layer class with required functions:
  - setup_weights_and_inputs (10 points)
  - weighted_sum_plus_bias (10 points) 
  - apply_activation_function (15 points)
  - calculate_loss (15 points)

### **Problem 2: Multi-Layer Neural Network Classification (50 points)**
Each team member selected and completed one classification problem:

**Problem 2a: Iris Dataset Classification** - *Completed by Kirk*
- 3-layer neural network for Iris species classification
- Multi-class classification with Softmax output

**Problem 2b: Breast Cancer Dataset Classification** - *Completed by Dallas*
- 3-layer neural network for tumor classification  
- Binary classification with Sigmoid output


**Total Points: 100**

---

## Problem 1: Dense Layer Implementation (50 points)

**Objective:** Develop a Dense_Layer class in Python with the following functions:
1. A function to setup/accept the inputs and weights (10 points)
2. A function to perform the weighted sum + bias (10 points)  
3. A function to perform the selected activation function (15 points)
4. A function to calculate the loss (predicted output vs target output) (15 points)


### Required Libraries for Problem 1

First, let's import the necessary libraries for our Dense Layer implementation:

In [ ]:
import numpy as np
import math

### Dense_Layer Class Implementation

Below is the complete implementation of the Dense_Layer class with all required functions:

In [ ]:
class Dense_Layer:
    def __init__(self, n_inputs, n_neurons, activation='relu'):
        """
        Initialize the Dense Layer
        
        Parameters:
        n_inputs: Number of input features
        n_neurons: Number of neurons in this layer
        activation: Activation function to use ('relu', 'sigmoid', 'tanh', 'linear')
        """
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        self.activation_type = activation
        
        # Initialize weights and biases
        self.weights = None
        self.biases = None
        self.inputs = None
        self.output = None
        
    def setup_weights_and_inputs(self, inputs, weights=None, biases=None):
        """
        Function to setup/accept the inputs and weights (10 points)
        
        Parameters:
        inputs: Input data (numpy array)
        weights: Weight matrix (if None, initialize randomly)
        biases: Bias vector (if None, initialize as zeros)
        """
        self.inputs = np.array(inputs)
        
        # Initialize weights if not provided
        if weights is None:
            # Xavier/Glorot initialization
            self.weights = np.random.randn(self.n_inputs, self.n_neurons) * np.sqrt(2.0 / self.n_inputs)
        else:
            self.weights = np.array(weights)
            
        # Initialize biases if not provided
        if biases is None:
            self.biases = np.zeros((1, self.n_neurons))
        else:
            self.biases = np.array(biases)
            
        print(f"Setup complete:")
        print(f"Input shape: {self.inputs.shape}")
        print(f"Weights shape: {self.weights.shape}")
        print(f"Biases shape: {self.biases.shape}")
        
    def weighted_sum_plus_bias(self):
        """
        Function to perform the weighted sum + bias (10 points)
        
        Returns:
        z: The linear combination (weighted sum + bias)
        """
        if self.inputs is None or self.weights is None or self.biases is None:
            raise ValueError("Must call setup_weights_and_inputs first!")
            
        # Compute weighted sum: inputs @ weights + bias
        z = np.dot(self.inputs, self.weights) + self.biases
        
        print(f"Weighted sum + bias computed. Output shape: {z.shape}")
        return z
        
    def apply_activation_function(self, z):
        """
        Function to perform the selected activation function (15 points)
        
        Parameters:
        z: Linear combination from weighted_sum_plus_bias
        
        Returns:
        activated_output: Output after applying activation function
        """
        if self.activation_type == 'relu':
            # ReLU: max(0, z)
            activated_output = np.maximum(0, z)
            
        elif self.activation_type == 'sigmoid':
            # Sigmoid: 1 / (1 + e^(-z))
            activated_output = 1 / (1 + np.exp(-np.clip(z, -500, 500)))  # Clip to prevent overflow
            
        elif self.activation_type == 'tanh':
            # Hyperbolic tangent
            activated_output = np.tanh(z)
            
        elif self.activation_type == 'softmax':
            # Softmax: e^(z_i) / sum(e^z) for multi-class classification
            exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
            activated_output = exp_z / np.sum(exp_z, axis=1, keepdims=True)
            
        elif self.activation_type == 'linear':
            # Linear activation (no change)
            activated_output = z
            
        else:
            raise ValueError(f"Unsupported activation function: {self.activation_type}")
            
        self.output = activated_output
        print(f"Applied {self.activation_type} activation function")
        return activated_output
        
    def calculate_loss(self, predicted_output, target_output, loss_type='mse'):
        """
        Function to calculate the loss (predicted output vs target output) (15 points)
        
        Parameters:
        predicted_output: The predicted values from the network
        target_output: The actual target values
        loss_type: Type of loss function ('mse', 'mae', 'binary_crossentropy', 'categorical_crossentropy')
        
        Returns:
        loss: The calculated loss value
        """
        predicted = np.array(predicted_output)
        target = np.array(target_output)
        
        if predicted.shape != target.shape:
            raise ValueError(f"Shape mismatch: predicted {predicted.shape} vs target {target.shape}")
            
        if loss_type == 'mse':
            # Mean Squared Error
            loss = np.mean((predicted - target) ** 2)
            
        elif loss_type == 'mae':
            # Mean Absolute Error
            loss = np.mean(np.abs(predicted - target))
            
        elif loss_type == 'binary_crossentropy':
            # Binary Cross-Entropy (for binary classification)
            # Clip predictions to prevent log(0)
            predicted_clipped = np.clip(predicted, 1e-15, 1 - 1e-15)
            loss = -np.mean(target * np.log(predicted_clipped) + (1 - target) * np.log(1 - predicted_clipped))
            
        elif loss_type == 'categorical_crossentropy':
            # Categorical Cross-Entropy (for multi-class classification)
            predicted_clipped = np.clip(predicted, 1e-15, 1 - 1e-15)
            loss = -np.mean(np.sum(target * np.log(predicted_clipped), axis=1))
            
        else:
            raise ValueError(f"Unsupported loss function: {loss_type}")
            
        print(f"Calculated {loss_type} loss: {loss:.6f}")
        return loss
        
    def forward_pass(self, inputs, target=None, loss_type='mse'):
        """
        Complete forward pass through the layer
        
        Parameters:
        inputs: Input data
        target: Target output (optional, for loss calculation)
        loss_type: Type of loss function to use
        
        Returns:
        output: Final output after activation
        loss: Loss value (if target provided)
        """
        # Setup inputs (weights should already be initialized)
        if self.weights is None:
            raise ValueError("Weights not initialized. Call setup_weights_and_inputs first.")
            
        self.inputs = np.array(inputs)
        
        # Forward pass
        z = self.weighted_sum_plus_bias()
        output = self.apply_activation_function(z)
        
        loss = None
        if target is not None:
            loss = self.calculate_loss(output, target, loss_type)
            
        return output, loss

### Problem 1: Function Testing and Demonstration

Let's test each function of our Dense_Layer class systematically to verify correct implementation:

In [ ]:
# TEST 1: Function setup_weights_and_inputs (10 points)
print("=== TEST 1: Setup Weights and Inputs Function ===")
print()

# Create a dense layer with 3 inputs and 2 neurons
layer1 = Dense_Layer(n_inputs=3, n_neurons=2, activation='relu')

# Sample input data (1 sample with 3 features)
sample_inputs = [[1.0, 2.0, 3.0]]

# Custom weights matrix (3x2) and biases (1x2)
custom_weights = [[0.2, 0.8], 
                  [0.5, -0.7], 
                  [0.1, 0.3]]
custom_biases = [[0.1, -0.2]]

print("Configuration:")
print(f"Number of inputs: {layer1.n_inputs}")
print(f"Number of neurons: {layer1.n_neurons}")
print(f"Input data: {sample_inputs}")
print(f"Custom weights: {custom_weights}")
print(f"Custom biases: {custom_biases}")
print()

# Setup the layer
print("Setting up layer...")
layer1.setup_weights_and_inputs(sample_inputs, custom_weights, custom_biases)
print()

print("✅ TEST 1 PASSED: setup_weights_and_inputs function working correctly!")
print("=" * 60)

In [ ]:
# TEST 2: Function weighted_sum_plus_bias (10 points)
print("=== TEST 2: Weighted Sum Plus Bias Function ===")
print()

print("Using the layer from TEST 1...")
print("Input:", sample_inputs[0])
print("Weights shape:", layer1.weights.shape)
print("Biases shape:", layer1.biases.shape)
print()

# Get weighted sum + bias
z = layer1.weighted_sum_plus_bias()
print("Calculated weighted sum + bias (z):", z.flatten())
print()

# Manual verification
print("Manual Verification:")
print("For neuron 1: (1.0×0.2) + (2.0×0.5) + (3.0×0.1) + 0.1 = 0.2 + 1.0 + 0.3 + 0.1 = 1.6")
print("For neuron 2: (1.0×0.8) + (2.0×-0.7) + (3.0×0.3) + (-0.2) = 0.8 - 1.4 + 0.9 - 0.2 = 0.1")
print()

# Verify our calculation
manual_calc = [1.6, 0.1]
print(f"Manual calculation: {manual_calc}")
print(f"Function result:    {z.flatten().tolist()}")

if np.allclose(z.flatten(), manual_calc):
    print("✅ TEST 2 PASSED: weighted_sum_plus_bias function working correctly!")
else:
    print("❌ TEST 2 FAILED: Results don't match manual calculation")

print("=" * 60)

In [ ]:
# TEST 3: Function apply_activation_function (15 points)
print("=== TEST 3: Apply Activation Function ===")
print()

# Using z from previous test
print("Input to activation function (z):", z.flatten())
print(f"Selected activation function: {layer1.activation_type}")
print()

# Apply activation function
output = layer1.apply_activation_function(z)
print("Output after ReLU activation:", output.flatten())
print()

print("Manual Verification:")
print("ReLU function: max(0, z)")
print("For z = [1.6, 0.1]:")
print("  max(0, 1.6) = 1.6")
print("  max(0, 0.1) = 0.1")
print("Expected output: [1.6, 0.1]")
print()

# Test different activation functions
print("Testing all activation functions:")
print("-" * 40)

test_z = [[-2.0, -0.5, 0.0, 0.5, 2.0]]
activations = ['relu', 'sigmoid', 'tanh', 'linear']

for activation in activations:
    test_layer = Dense_Layer(n_inputs=1, n_neurons=5, activation=activation)
    test_output = test_layer.apply_activation_function(np.array(test_z))
    print(f"{activation.upper():8} | Input: {test_z[0]} → Output: {test_output.flatten().round(3).tolist()}")

print()
print("✅ TEST 3 PASSED: apply_activation_function working for all activation types!")
print("=" * 60)

In [ ]:
# TEST 4: Function calculate_loss (15 points)
print("=== TEST 4: Calculate Loss Function ===")
print()

# Using output from previous test as predictions
predictions = output
target = [[1.5, 0.0]]

print("Configuration:")
print(f"Predicted output: {predictions.flatten()}")
print(f"Target output:    {target[0]}")
print()

# Calculate MSE loss
loss = layer1.calculate_loss(predictions, target, 'mse')
print(f"MSE Loss: {loss:.6f}")
print()

# Manual verification of MSE
print("Manual MSE Verification:")
pred_flat = predictions.flatten()
target_flat = np.array(target).flatten()
manual_mse = np.mean((pred_flat - target_flat) ** 2)
print(f"Manual calculation: mean([({pred_flat[0]:.1f} - {target_flat[0]:.1f})², ({pred_flat[1]:.1f} - {target_flat[1]:.1f})²])")
print(f"                  = mean([({pred_flat[0] - target_flat[0]:.1f})², ({pred_flat[1] - target_flat[1]:.1f})²])")
print(f"                  = mean([{(pred_flat[0] - target_flat[0])**2:.2f}, {(pred_flat[1] - target_flat[1])**2:.2f}])")
print(f"                  = {manual_mse:.6f}")
print()

# Test different loss functions
print("Testing all loss functions:")
print("-" * 50)

# Test cases for different loss types
test_pred = [[0.8, 0.2]]
test_target = [[1.0, 0.0]]

loss_functions = ['mse', 'mae', 'categorical_crossentropy']
for loss_type in loss_functions:
    test_loss = layer1.calculate_loss(test_pred, test_target, loss_type)
    print(f"{loss_type.upper():20} | Loss: {test_loss:.6f}")

# Binary classification test
binary_pred = [[0.7]]
binary_target = [[1.0]]
binary_loss = layer1.calculate_loss(binary_pred, binary_target, 'binary_crossentropy')
print(f"{'BINARY_CROSSENTROPY':20} | Loss: {binary_loss:.6f}")

print()
print("✅ TEST 4 PASSED: calculate_loss function working for all loss types!")
print("=" * 60)

In [ ]:
# COMPREHENSIVE TEST: Complete Forward Pass Integration
print("=== COMPREHENSIVE TEST: Complete Forward Pass ===")
print()

# Create a new layer for batch processing
batch_layer = Dense_Layer(n_inputs=4, n_neurons=3, activation='relu')

# Sample batch of data (3 samples, 4 features each)
batch_inputs = [[1.0, 2.0, -1.0, 0.5],
                [0.5, -1.0, 2.0, 1.5],
                [-0.5, 1.0, 0.0, -1.0]]

# Target outputs for the batch
target_outputs = [[1.0, 0.0, 0.5],
                  [0.0, 1.0, 0.3],
                  [0.5, 0.5, 1.0]]

print("Configuration:")
print(f"Input shape: {np.array(batch_inputs).shape} (3 samples, 4 features)")
print(f"Target shape: {np.array(target_outputs).shape} (3 samples, 3 outputs)")
print(f"Layer: {batch_layer.n_inputs} inputs → {batch_layer.n_neurons} neurons")
print(f"Activation: {batch_layer.activation_type}")
print()

# Initialize with random weights (Xavier initialization)
batch_layer.setup_weights_and_inputs(batch_inputs)
print()

print("Running complete forward pass:")
print("-" * 30)

# Step-by-step forward pass
z = batch_layer.weighted_sum_plus_bias()
print(f"1. Weighted sum + bias (z) shape: {z.shape}")
print(f"   Sample outputs: {z[0].round(3)}")

output = batch_layer.apply_activation_function(z)
print(f"2. After ReLU activation shape: {output.shape}")
print(f"   Sample outputs: {output[0].round(3)}")

loss = batch_layer.calculate_loss(output, target_outputs, 'mse')
print(f"3. MSE Loss: {loss:.6f}")
print()

print("FINAL RESULTS SUMMARY:")
print("=" * 50)
print("Sample | Input Features           | Predicted Output      | Target Output        | Individual Error")
print("-------|--------------------------|----------------------|----------------------|------------------")

for i, (inp, pred, target) in enumerate(zip(batch_inputs, output, target_outputs)):
    individual_error = np.mean((np.array(pred) - np.array(target)) ** 2)
    print(f"   {i+1}   | {str(inp):24} | {str(pred.round(3)):20} | {str(target):20} | {individual_error:.6f}")

print()
print(f"Overall MSE Loss: {loss:.6f}")
print()
print("🎉 COMPREHENSIVE TEST PASSED: All Dense Layer functions working correctly!")
print("=" * 60)

---

## Problem 2: Multi-Layer Neural Network Classification (50 points)

**Objective:** Use the Dense_Layer class from Problem 1 to build complete multi-layer neural networks for classification tasks.

**Team Assignment:** Each member completed one classification problem:
- **Problem 2a:** Iris Dataset Classification - *Kirk Henrich Gamo*
- **Problem 2b:** Breast Cancer Dataset Classification - *Dallas Aquino*

---

### Multi-Layer Network Class

The following class uses Dense_Layer instances from Problem 1 to create complete neural networks:

In [ ]:
# Multi-layer network class using Dense_Layer from Problem 1
import numpy as np

class MultiLayerNetwork:
    """
    Multi-layer neural network using Dense_Layer class from Problem 1
    This demonstrates how the Dense_Layer class can be used to build complete neural networks
    """
    def __init__(self):
        self.layers = []
        
    def add_dense_layer(self, n_inputs, n_neurons, activation, weights=None, biases=None):
        """Add a Dense_Layer instance to the network"""
        layer = Dense_Layer(n_inputs, n_neurons, activation)
        if weights is not None and biases is not None:
            # Initialize with provided weights and biases
            dummy_input = [[0] * n_inputs]  # Dummy input for setup
            layer.setup_weights_and_inputs(dummy_input, weights, biases)
            
        # Initialize additional attributes needed for tracking
        layer.current_weighted_sums = None
        layer.layer_outputs = None
        
        self.layers.append(layer)
        return layer
    
    def forward_pass(self, X):
        """Perform forward pass through all Dense_Layer instances"""
        current_input = np.array(X)
        
        for i, layer in enumerate(self.layers):
            # Update the layer's inputs
            layer.inputs = current_input
            
            # Compute weighted sum + bias using Dense_Layer method
            z = layer.weighted_sum_plus_bias()
            layer.current_weighted_sums = z  # Store for later access
            
            # Apply activation function using Dense_Layer method
            output = layer.apply_activation_function(z)
            layer.layer_outputs = output  # Store for later access
            
            # Output becomes input for next layer
            current_input = output
            
        return current_input

print("✅ Multi-layer network class defined using Dense_Layer from Problem 1!")
print("This implementation demonstrates how the Dense_Layer class can build complete neural networks.")

---

### Problem 2a: Iris Dataset Classification
### *Completed by: Kirk Henrich Gamo*

**Objective:** Given inputs from the Iris Dataset using sepal length, sepal width, petal length and petal width,determine what class (Iris-setosa, Iris-versicolor, and Iris-virginica) the input belongs to by calculating the network output through a 3-layer neural network using Dense_Layer class.

**Network Architecture:**
- **Input Layer:** 4 features (sepal length, sepal width, petal length, petal width)
- **Hidden Layer 1:** 3 neurons with ReLU activation
- **Hidden Layer 2:** 2 neurons with Sigmoid activation
- **Output Layer:** 3 neurons (one for each iris species) with softmax activation

### Problem 2a: Network Configuration and Setup

Setting up the Iris classification problem with the given network architecture:

In [ ]:
# PROBLEM 2a SETUP: Iris Classification using Dense_Layer Class 
print("=== PROBLEM 2a SETUP: IRIS CLASSIFICATION ===")
print("Implemented by: Kirk Henrich Gamo")
print()

# Input data from Iris Dataset (sepal length, sepal width, petal length, petal width)
# Based on the picture: X = [5.1, 3.5, 1.4, 0.2]
X_input = [[5.1, 3.5, 1.4, 0.2]]  # Sample Iris measurements

# Target output from the picture: [0.7, 0.2, 0.1]
target_output = [[0.7, 0.2, 0.1]]  # Expected output for this sample

print("Problem Configuration:")
print(f"Input features (X): {X_input[0]}")
print("  - Sepal Length: {:.1f} cm".format(X_input[0][0]))
print("  - Sepal Width:  {:.1f} cm".format(X_input[0][1]))
print("  - Petal Length: {:.1f} cm".format(X_input[0][2]))
print("  - Petal Width:  {:.1f} cm".format(X_input[0][3]))
print()
print(f"Target output: {target_output[0]} (Expected: [setosa, versicolor, virginica])")
print()

# Network Architecture: 4 → 3 → 2 → 3 neurons
print("Network Architecture:")
print("Input Layer (4) → Hidden Layer 1 (3) → Hidden Layer 2 (2) → Output Layer (3)")
print()


# First Hidden Layer: W1 (4x3) and B1 (1x3) - ReLU activation 
W1 = [[ 0.2,  0.5, -0.3],
      [ 0.1, -0.2,  0.4],
      [-0.4,  0.3,  0.2],
      [ 0.6, -0.1,  0.5]]
B1 = [[3.0, -2.1, 0.6]]

# Second Hidden Layer: W2 (3x2) and B2 (1x2) - Sigmoid activation 
W2 = [[ 0.3, -0.5],
      [ 0.7,  0.2],
      [-0.6,  0.4]]
B2 = [[4.3, 6.4]]

# Output Layer: W3 (2x3) and B3 (1x3) - Softmax activation 
W3 = [[ 0.5, -0.3,  0.8],
      [-0.2,  0.6, -0.4]]
B3 = [[-1.5, 2.1, -3.3]]

print("Layer Configurations (CORRECTED from the picture):")
print(f"Layer 1: {np.array(W1).shape} weights, {np.array(B1).shape} biases, ReLU activation")
print(f"Layer 2: {np.array(W2).shape} weights, {np.array(B2).shape} biases, Sigmoid activation")
print(f"Layer 3: {np.array(W3).shape} weights, {np.array(B3).shape} biases, Softmax activation")
print()

# Create network using Dense_Layer class
network = MultiLayerNetwork()

# Add layers using Dense_Layer instances with CORRECT dimensions
layer1 = network.add_dense_layer(4, 3, 'relu', W1, B1)
layer2 = network.add_dense_layer(3, 2, 'sigmoid', W2, B2)
layer3 = network.add_dense_layer(2, 3, 'softmax', W3, B3)

print("✅ Network created successfully using Dense_Layer class from Problem 1!")
print(f"Number of Dense_Layer instances: {len(network.layers)}")
print()

### Problem 2a: Forward Pass Execution

Running the complete forward pass through the multi-layer network:

In [ ]:
# PROBLEM 2a FORWARD PASS: Complete Network Computation using Dense_Layer Class
print("=== PROBLEM 2a FORWARD PASS COMPUTATION ===")
print()

print("Input sample:", X_input[0])
print()

# Forward pass through all Dense_Layer instances
final_output = network.forward_pass(X_input)

print("Layer-by-Layer Computation Using Dense_Layer Class:")
print("=" * 55)

# Layer 1 computation
print("LAYER 1 (4 → 3 neurons, ReLU activation):")
layer1_output = network.layers[0].layer_outputs
print(f"Input: {network.layers[0].inputs.flatten()}")
print(f"Weighted Sum: {network.layers[0].current_weighted_sums.flatten()}")
print(f"After ReLU: {layer1_output.flatten()}")
print("→ Passed to Layer 2")
print()

# Layer 2 computation  
print("LAYER 2 (3 → 2 neurons, Sigmoid activation):")
layer2_output = network.layers[1].layer_outputs
print(f"Input: {network.layers[1].inputs.flatten()}")
print(f"Weighted Sum: {network.layers[1].current_weighted_sums.flatten()}")
print(f"After Sigmoid: {layer2_output.flatten()}")
print("→ Passed to Layer 3")
print()

# Layer 3 computation (Final output)
print("LAYER 3 (2 → 3 neurons, Softmax activation - FINAL OUTPUT):")
layer3_output = network.layers[2].layer_outputs
print(f"Input: {network.layers[2].inputs.flatten()}")
print(f"Weighted Sum: {network.layers[2].current_weighted_sums.flatten()}")
print(f"After Softmax: {layer3_output.flatten()}")
print()

# Since layer 3 now uses softmax activation, the outputs are already probabilities
final_probs = layer3_output.flatten()
predicted_class = np.argmax(final_probs)
class_names = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

print("=" * 55)
print("FINAL CLASSIFICATION RESULTS:")
print("Class Probabilities (from Softmax layer):")
for i, (class_name, prob) in enumerate(zip(class_names, final_probs)):
    marker = " ← PREDICTED" if i == predicted_class else ""
    print(f"  {class_name:15}: {prob:.4f} ({prob*100:.1f}%){marker}")

print()
print(f"Predicted Class: {class_names[predicted_class]}")
print(f"Actual Class:    {class_names[np.argmax(target_output[0])]}")
print(f"Classification: {'✅ Correct' if predicted_class == np.argmax(target_output[0]) else '❌ Incorrect'}")
print()

### Problem 2a: Verification and Loss Calculation

Manual step-by-step verification using individual Dense_Layer instances:

In [ ]:
# PROBLEM 2a VERIFICATION: Manual Step-by-Step Calculation using Dense_Layer Class
print("=== PROBLEM 2a VERIFICATION: MANUAL CALCULATION ===")
print("Verifying Dense_Layer implementation step by step")
print()

# Manual verification using Dense_Layer class functions
print("MANUAL VERIFICATION USING DENSE_LAYER CLASS:")
print("=" * 50)

# Create individual layers for verification with CORRECT dimensions
layer1_verify = Dense_Layer(4, 3, 'relu')
layer2_verify = Dense_Layer(3, 2, 'sigmoid')  
layer3_verify = Dense_Layer(2, 3, 'softmax')

# Layer 1 verification
print("LAYER 1 VERIFICATION (4 → 3, ReLU):")
layer1_verify.setup_weights_and_inputs(X_input, W1, B1)
z1 = layer1_verify.weighted_sum_plus_bias()
a1 = layer1_verify.apply_activation_function(z1)
print(f"✓ Input: {layer1_verify.inputs.flatten()}")
print(f"✓ Weighted Sum: {z1.flatten()}")
print(f"✓ ReLU Output: {a1.flatten()}")
print()

# Layer 2 verification  
print("LAYER 2 VERIFICATION (3 → 2, Sigmoid):")
layer2_verify.setup_weights_and_inputs(a1, W2, B2)
z2 = layer2_verify.weighted_sum_plus_bias()
a2 = layer2_verify.apply_activation_function(z2)
print(f"✓ Input: {layer2_verify.inputs.flatten()}")
print(f"✓ Weighted Sum: {z2.flatten()}")
print(f"✓ Sigmoid Output: {a2.flatten()}")
print()

# Layer 3 verification (Final)
print("LAYER 3 VERIFICATION (2 → 3, Softmax - FINAL):")
layer3_verify.setup_weights_and_inputs(a2, W3, B3)
z3 = layer3_verify.weighted_sum_plus_bias()
a3 = layer3_verify.apply_activation_function(z3)
print(f"✓ Input: {layer3_verify.inputs.flatten()}")
print(f"✓ Weighted Sum: {z3.flatten()}")
print(f"✓ Final Output (Softmax): {a3.flatten()}")
print()

# Compare with network results
print("=" * 50)
print("COMPARISON WITH NETWORK RESULTS:")
manual_result = a3.flatten()
network_result = layer3_output.flatten()

print(f"Manual calculation:  {manual_result}")
print(f"Network calculation: {network_result}")
print(f"Difference: {np.abs(manual_result - network_result)}")
print(f"Match: {'✅ PERFECT MATCH' if np.allclose(manual_result, network_result, atol=1e-6) else '❌ MISMATCH'}")
print()

# Loss calculation using Dense_Layer class
print("LOSS CALCULATION:")
loss_categorical = layer3_verify.calculate_loss(a3, target_output, 'categorical_crossentropy')
loss_mse = layer3_verify.calculate_loss(a3, target_output, 'mse')
print(f"Categorical Cross-Entropy Loss: {loss_categorical:.6f}")
print(f"Mean Squared Error (MSE) Loss: {loss_mse:.6f}")
print(f"Target: {target_output[0]}, Prediction: {a3.flatten()}")
print()
print("✅ Problem 2a completed successfully using Dense_Layer class from Problem 1!")

---

### Problem 2b: Breast Cancer Dataset Classification
### *Completed by: Dallas Aquino*

**Objective:** Given inputs from the Breast Cancer Dataset using three features (Mean Radius, Mean Texture, and Mean Smoothness), determine whether the tumor is Benign (0) or Malignant (1) by calculating the network outputs step by step.

**Network Architecture:**
- **Input Layer:** 3 features (Mean Radius, Mean Texture, Mean Smoothness)
- **Hidden Layer 1:** 3 neurons with ReLU activation
- **Hidden Layer 2:** 2 neurons with Sigmoid activation
- **Output Layer:** 1 neuron (Binary classification) with Sigmoid activation


### Problem 2b: Network Configuration and Setup

Setting up the Breast Cancer classification problem with the given network architecture:

In [ ]:
# PROBLEM 2b SETUP: Breast Cancer Classification using Dense_Layer Class
print("=== PROBLEM 2b SETUP: BREAST CANCER CLASSIFICATION ===")
print("Implemented by: Dallas Aquino")
print()

# Input data from Breast Cancer Dataset (Mean Radius, Mean Texture, Mean Smoothness)
X_breast = [[14.1, 20.3, 0.095]]  # Sample breast cancer measurements

# Target output (Binary classification: 0 = Benign, 1 = Malignant)
target_breast = [[1]]  # This sample should be Malignant (1)

print("Problem Configuration:")
print(f"Input features (X): {X_breast[0]}")
print("  - Mean Radius:     {:.1f}".format(X_breast[0][0]))
print("  - Mean Texture:    {:.1f}".format(X_breast[0][1]))
print("  - Mean Smoothness: {:.3f}".format(X_breast[0][2]))
print()
print(f"Target output: {target_breast[0][0]} (1 = Malignant)")
print()

# Network Architecture: 3 → 3 → 2 → 1 neurons
print("Network Architecture:")
print("Input Layer (3) → Hidden Layer 1 (3) → Hidden Layer 2 (2) → Output Layer (1)")
print()

# Define weights and biases for each layer EXACTLY as shown in the picture
# Layer 1: 3 inputs → 3 neurons (ReLU) 
W1_breast = [[ 0.5, -0.3,  0.8],
             [ 0.2,  0.4, -0.6], 
             [-0.7,  0.9,  0.1]]
B1_breast = [[0.3, -0.5, 0.6]]

# Layer 2: 3 inputs → 2 neurons (Sigmoid) - FIXED DIMENSIONS
W2_breast = [[ 0.6, -0.3],
             [-0.2,  0.5],
             [ 0.4,  0.7]]
B2_breast = [[0.1, -0.8]]

# Layer 3: 2 inputs → 1 neuron (Sigmoid) - FIXED DIMENSIONS
W3_breast = [[0.7], 
             [-0.5]]
B3_breast = [[0.2]]

print("Layer Configurations:")
print(f"Layer 1: {np.array(W1_breast).shape} weights, {np.array(B1_breast).shape} biases, ReLU activation")
print(f"Layer 2: {np.array(W2_breast).shape} weights, {np.array(B2_breast).shape} biases, Sigmoid activation")
print(f"Layer 3: {np.array(W3_breast).shape} weights, {np.array(B3_breast).shape} biases, Sigmoid activation")
print()

# Create network using Dense_Layer class
network_breast = MultiLayerNetwork()

# Add Dense_Layer instances with CORRECT dimensions from picture
layer1_breast = network_breast.add_dense_layer(3, 3, 'relu', W1_breast, B1_breast)
layer2_breast = network_breast.add_dense_layer(3, 2, 'sigmoid', W2_breast, B2_breast)
layer3_breast = network_breast.add_dense_layer(2, 1, 'sigmoid', W3_breast, B3_breast)

print("✅ Network created successfully using Dense_Layer class from Problem 1!")
print(f"Number of Dense_Layer instances: {len(network_breast.layers)}")
print()

### Problem 2b: Forward Pass Execution

Running the complete forward pass through the multi-layer Breast Cancer classification network:

In [ ]:
# PROBLEM 2b FORWARD PASS: Complete Network Computation using Dense_Layer Class
print("=== PROBLEM 2b FORWARD PASS COMPUTATION ===")
print()

print("Input sample:", X_breast[0])
print()

# Forward pass through all Dense_Layer instances
final_output_breast = network_breast.forward_pass(X_breast)

print("Layer-by-Layer Computation Using Dense_Layer Class:")
print("=" * 55)

# Layer 1 computation
print("LAYER 1 (3 → 3 neurons, ReLU activation):")
layer1_output_breast = network_breast.layers[0].layer_outputs
print(f"Input: {network_breast.layers[0].inputs.flatten()}")
print(f"Weighted Sum: {network_breast.layers[0].current_weighted_sums.flatten()}")
print(f"After ReLU: {layer1_output_breast.flatten()}")
print("→ Passed to Layer 2")
print()

# Layer 2 computation  
print("LAYER 2 (3 → 2 neurons, Sigmoid activation):")
layer2_output_breast = network_breast.layers[1].layer_outputs
print(f"Input: {network_breast.layers[1].inputs.flatten()}")
print(f"Weighted Sum: {network_breast.layers[1].current_weighted_sums.flatten()}")
print(f"After Sigmoid: {layer2_output_breast.flatten()}")
print("→ Passed to Layer 3")
print()

# Layer 3 computation (Final output)
print("LAYER 3 (2 → 1 neuron, Sigmoid activation - FINAL OUTPUT):")
layer3_output_breast = network_breast.layers[2].layer_outputs
print(f"Input: {network_breast.layers[2].inputs.flatten()}")
print(f"Weighted Sum: {network_breast.layers[2].current_weighted_sums.flatten()}")
print(f"After Sigmoid: {layer3_output_breast.flatten()}")
print()

print("=" * 55)
print("FINAL RESULTS:")
final_prediction_breast = layer3_output_breast[0][0]
print(f"Network Prediction: {final_prediction_breast:.6f}")
print(f"Target Value: {target_breast[0][0]}")
print(f"Prediction Interpretation: {'Malignant' if final_prediction_breast > 0.5 else 'Benign'}")
print(f"Classification: {'✅ Correct' if (final_prediction_breast > 0.5) == (target_breast[0][0] == 1) else '❌ Incorrect'}")
print()

### Problem 2b: Verification and Loss Calculation

Manual step-by-step verification using individual Dense_Layer instances:

In [ ]:
# PROBLEM 2b VERIFICATION: Manual Step-by-Step Calculation using Dense_Layer Class
print("=== PROBLEM 2b VERIFICATION: MANUAL CALCULATION ===")
print("Verifying Dense_Layer implementation step by step")
print()

# Manual verification using Dense_Layer class functions
print("MANUAL VERIFICATION USING DENSE_LAYER CLASS:")
print("=" * 50)

# Create individual layers for verification with CORRECT dimensions
layer1_verify = Dense_Layer(3, 3, 'relu')
layer2_verify = Dense_Layer(3, 2, 'sigmoid')  
layer3_verify = Dense_Layer(2, 1, 'sigmoid')

# Layer 1 verification
print("LAYER 1 VERIFICATION (3 → 3, ReLU):")
layer1_verify.setup_weights_and_inputs(X_breast, W1_breast, B1_breast)
z1 = layer1_verify.weighted_sum_plus_bias()
a1 = layer1_verify.apply_activation_function(z1)
print(f"✓ Input: {layer1_verify.inputs.flatten()}")
print(f"✓ Weighted Sum: {z1.flatten()}")
print(f"✓ ReLU Output: {a1.flatten()}")
print()

# Layer 2 verification  
print("LAYER 2 VERIFICATION (3 → 2, Sigmoid):")
layer2_verify.setup_weights_and_inputs(a1, W2_breast, B2_breast)
z2 = layer2_verify.weighted_sum_plus_bias()
a2 = layer2_verify.apply_activation_function(z2)
print(f"✓ Input: {layer2_verify.inputs.flatten()}")
print(f"✓ Weighted Sum: {z2.flatten()}")
print(f"✓ Sigmoid Output: {a2.flatten()}")
print()

# Layer 3 verification (Final)
print("LAYER 3 VERIFICATION (2 → 1, Sigmoid - FINAL):")
layer3_verify.setup_weights_and_inputs(a2, W3_breast, B3_breast)
z3 = layer3_verify.weighted_sum_plus_bias()
a3 = layer3_verify.apply_activation_function(z3)
print(f"✓ Input: {layer3_verify.inputs.flatten()}")
print(f"✓ Weighted Sum: {z3.flatten()}")
print(f"✓ Final Output: {a3.flatten()}")
print()

# Compare with network results
print("=" * 50)
print("COMPARISON WITH NETWORK RESULTS:")
manual_result = a3[0][0]
network_result = final_prediction_breast  # From previous cell

print(f"Manual calculation:  {manual_result:.8f}")
print(f"Network calculation: {network_result:.8f}")
print(f"Difference: {abs(manual_result - network_result):.10f}")
print(f"Match: {'✅ PERFECT MATCH' if abs(manual_result - network_result) < 1e-6 else '❌ MISMATCH'}")
print()

# Loss calculation using Dense_Layer class
print("LOSS CALCULATION:")
loss_binary = layer3_verify.calculate_loss(a3, target_breast, 'binary_crossentropy')
loss_mse = layer3_verify.calculate_loss(a3, target_breast, 'mse')
print(f"Binary Cross-Entropy Loss: {loss_binary:.6f}")
print(f"Mean Squared Error (MSE) Loss: {loss_mse:.6f}")
print(f"Target: {target_breast[0][0]}, Prediction: {a3[0][0]:.6f}")
print()
print("✅ Problem 2b completed successfully using Dense_Layer class from Problem 1!")